In [2]:
import os
import re
import pdfplumber
import pandas as pd
import camelot

In [ ]:
# for filename in os.listdir(pdf_dir):
#     if filename.lower().endswith(".pdf"):
#         pdf_path = os.path.join(pdf_dir, filename)
#         with pdfplumber.open(pdf_path) as pdf:
#             all_text = ""
#             preserved_layout_text = ""
#             body_lines = []
            
#             for page in pdf.pages:
#                 all_text += page.extract_text() + "\n" or ""
#                 # Use preserved_text with extra parameters to preserve spacing
#                 # preserved_text = "" 
#                 # preserved_text += page.extract_text(
#                 #     layout=True,      # Preserves layout including spaces
#                 #     keep_blank_chars=True,  # Keeps blank characters
#                 #     x_tolerance=1,    # Smaller tolerance preserves spacing better
#                 #     y_tolerance=1
#                 # ) or ""
#                 # preserved_layout_text += preserved_text + "\n"
#                 # print(f"-----All text:\n{all_text}\n-----")
#                 # print(f"-----Preserved Layout Text:\n{preserved_layout_text}\n-----")

#                 # BODY TEXT EXTRACTION
#                 lines = all_text.split('\n')
#                 # More precise header pattern that matches your document's header format
#                 header_pattern = re.compile(r'^\*\d{2}B\d+\*$')  # Pattern like "*05B00041981*"
#                 in_header = False
#                 for line in lines:
#                     # Check for header pattern
#                     if header_pattern.match(line.strip()):
#                         in_header = True
#                         continue
#                     # Skip the line immediately after header pattern
#                     if in_header and line.strip().startswith('Order '):
#                         in_header = False
#                         continue
#                     # Skip footer lines
#                     if re.match(r'^Page \d+ of \d+$', line.strip()):
#                         continue
#                     body_lines.append(line)
                
#                 body_text = '\n'.join(body_lines)
#                 # Remove any remaining blank lines at start/end
#                 body_text = body_text.strip()
#                 # print(f"{body_text}")

#                 # #EXTRACT ORDER DETAILS LINES
#                 order_details_lines = []
#                 capture = False
#                 for line in body_text.split('\n'):
#                     if "This document contains certified products. Please see table below for details." in line:
#                         capture = True
#                         continue
#                     if "Applicable Certifications" in line:
#                         capture = False
#                         continue
#                     if capture and line.strip():
#                         order_details_lines.append(line)
                
#                 # Join the order details lines and clean up
#                 order_details_text = '\n'.join(order_details_lines).strip()
#                 print(order_details_text)
                
#                 #Camelot extraction
#                 tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
#                 camelot_tables = []
#                 for table in tables:
#                     camelot_df = table.df
#                     camelot_tables.append(camelot_df)
#                     # print(f"----\n{camelot_df.to_string(index=False)}\n-----")

In [3]:
pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Other Samples"
# pdf_dir = r"C:\Users\Altersense\Desktop\ERP-RPA\Sample\Main"

for filename in os.listdir(pdf_dir):
    if filename.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_dir, filename)

        with pdfplumber.open(pdf_path) as pdf:
            all_text = ""           # Raw extracted text from all pages
            all_pages_body = []     # Store processed body text per page

            # Header pattern like "*05B00041981*"
            header_pattern = re.compile(r'^\*\d{2}B\d+\*$')

            # === PAGE LOOP ===
            for page in pdf.pages:
                # Append raw text to all_text (full PDF content, unprocessed)
                page_text = page.extract_text() or ""
                all_text += page_text + "\n"

                # Clean body text for this page
                lines = page_text.split('\n')
                in_header = False
                page_body_lines = []

                for line in lines:
                    if header_pattern.match(line.strip()):
                        in_header = True
                        continue
                    if in_header and line.strip().startswith('Order '):
                        in_header = False
                        continue
                    if re.match(r'^Page \d+ of \d+$', line.strip()):
                        continue
                    page_body_lines.append(line)

                cleaned_body = '\n'.join(page_body_lines).strip()
                all_pages_body.append(cleaned_body)

            # === Combine cleaned page texts ===
            body_text = "\n".join(all_pages_body)

            # === Camelot table extraction (run ONCE per file) ===
            tables = camelot.read_pdf(pdf_path, pages='all', flavor='stream')
            camelot_tables = [table.df for table in tables]

            # === Extract order details ===
            order_details_lines = []
            capture = False
            last_total_usd_index = None
            lines = body_text.split("\n")

            # Pre-calc last "Total USD" if "Applicable Certifications" not found
            if "Applicable Certifications" not in body_text:
                for i, line in enumerate(lines):
                    if "Total USD" in line:
                        last_total_usd_index = i

            for i, line in enumerate(lines):
                if "This document contains certified products. Please see table below for details." in line:
                    capture = True
                    continue
                
                if not capture and "shipment FOB date" in line:
                    capture = True

                if "Applicable Certifications" in line:
                    capture = False
                    continue

                # Stop continuing if we hit the last "Total USD" line
                if last_total_usd_index is not None and i > last_total_usd_index:
                    capture = False

                if capture and line.strip():
                    order_details_lines.append(line)

            order_details_text = "\n".join(order_details_lines)


            print(f"All Text:\n{all_text}")  # Raw text
            print(f"Body Text:\n{body_text}")  # Cleaned text without headers/footers
            print(f"Order Details:\n{order_details_text}")  # Extracted order details texts  

All Text:
*05B00039218*
Order 05B00039218
Buy-from Vendor No. 9900960321
MANAMI FASHIONS LTD.
KABIRPUR, ASHULIA, Order Date 07-30-2024 (MM-DD-YYYY)
SAVAR, Purchaser Paul Zwillus
Dhaka, 1349 E-Mail paul.zwillus@brands-fashion.com
Dhaka
Phone No. +49 4181 / 9422 - 208
Bangladesh
Payment Terms payment after receipt of documen
Payment Method letter of credit
Agency to Address Ship-to Address Transport Method SEA
TRITON TEXTILE LIMITED Bangladesh BRANDS Fashion GmbH Shipping Agent Code OGL
House No. 20 (1st + 2nd & 3rd Floor) Lager Buchholz Port of Departure CGP
Road No. 99, Gulshan-02 Müllerstr. 11 Shipment Date ETD 11-10-2024 (MM-DD-YYYY)
Dhaka 21244 Buchholz Cost Center OBI
Bangladesh Germany
Comment: Lageraufstockung Polos
OBI267-SET 3 STK 3 x OBI Classic Mens Polo shipment FOB date Price 17,70
shortsleeve Embroidery
anthrazit/orange Customs no.: 6105 1000 00
INTERNAT1
Size XS S M L XL XXL 3XL 4XL 5XL
Qty 1.200 1.100 250 80 120 10
Total 2.760 PIECES
Amount 48.852,00 USD
OBI269-SET 3 STK

In [5]:
# Camelot Extractions with Debugging and Fallback
ship_to_address = 'Not found'
agency_to_address = "Not found"
buying_house_address = "Not found"
agency_full_address = "Not found"

print(f"\n=== Camelot Debug ===")
print(f"Total tables found: {len(camelot_tables)}")

found_table = None
found_row = None
found_col = None

# First pass: Try to find the structured address table
for i, df in enumerate(camelot_tables):
    print(f"\nTable {i+1}: Shape {df.shape}")
    print("First 2 rows preview:")
    print(df.head(2))  # Show first 2 rows
    
    if df.shape[1] >= 2:
        first_cell = str(df.iloc[0, 0])
        print(f"First cell content: '{first_cell}'")
        
        if "Ship-to Address" in first_cell:
            print("--> Found address table in standard position")
            ship_to_address = df.iloc[1, 0].strip()
            agency_to_address = df.iloc[1, 1].strip()
            
            buying_house_address = "\n".join(
                [row[0].strip() for row in df.iloc[2:].values if row[0].strip()]
            )
            agency_full_address = "\n".join(
                [row[1].strip() for row in df.iloc[2:].values if row[1].strip()]
            )
            found_table = df
            break

# Second pass: Fallback search through all tables if not found
if found_table is None:
    print("\nStandard address table not found, initiating fallback search...")
    for i, df in enumerate(camelot_tables):
        print(f"\nSearching Table {i+1} for address patterns...")
        
        # Search through all cells for "Ship-to Address"
        for row_idx in range(df.shape[0]):
            for col_idx in range(df.shape[1]):
                cell_value = str(df.iloc[row_idx, col_idx]).strip()
                if "Ship-to Address" in cell_value:
                    print(f"Found 'Ship-to Address' at Table {i+1}, Row {row_idx+1}, Column {col_idx+1}")
                    found_table = df
                    found_row = row_idx
                    found_col = col_idx
                    break
            if found_table is not None:
                break
        if found_table is not None:
            break

# Extract from found position if located
if found_table is not None:
    if found_row is not None:  # Found in fallback search
        print("\nExtracting from fallback position...")
        # Get the column with "Ship-to Address" label
        ship_col = found_col
        
        # Try to find agency column - look for "Agency" in same row
        agency_col = None
        for col_idx in range(found_table.shape[1]):
            cell_value = str(found_table.iloc[found_row, col_idx]).strip()
            if "Agency" in cell_value and col_idx != ship_col:
                agency_col = col_idx
                break
        
        # Extract ship-to address (next row in same column)
        if found_row + 1 < found_table.shape[0]:
            ship_to_address = str(found_table.iloc[found_row + 1, ship_col]).strip()
            
            # Extract buying house address (subsequent rows in same column)
            buying_house_rows = []
            for row_idx in range(found_row + 2, found_table.shape[0]):
                cell_value = str(found_table.iloc[row_idx, ship_col]).strip()
                if cell_value:
                    buying_house_rows.append(cell_value)
            buying_house_address = "\n".join(buying_house_rows)
        
        # Extract agency information if agency column found
        if agency_col is not None:
            if found_row + 1 < found_table.shape[0]:
                agency_to_address = str(found_table.iloc[found_row + 1, agency_col]).strip()
            
            # Extract full agency address
            agency_rows = []
            for row_idx in range(found_row + 2, found_table.shape[0]):
                cell_value = str(found_table.iloc[row_idx, agency_col]).strip()
                if cell_value:
                    agency_rows.append(cell_value)
            agency_full_address = "\n".join(agency_rows)

print("\nFinal Address Values:")
print(f"Ship-to: {ship_to_address}")
print(f"Agency: {agency_to_address}")
print(f"Buying House: {buying_house_address}")
print(f"Full Agency: {agency_full_address}")


=== Camelot Debug ===
Total tables found: 3

Table 1: Shape (23, 4)
First 2 rows preview:
  0 1 2                   3
0             *05B00039218*
1        Order  05B00039218
First cell content: ''

Table 2: Shape (8, 12)
First 2 rows preview:
  0  1  2  3  4  5  6  7  8  9                   10 11
0                                     *05B00039218*   
1                                Order  05B00039218   
First cell content: ''

Table 3: Shape (8, 11)
First 2 rows preview:
  0  1  2  3  4  5  6  7  8                   9  10
0                                  *05B00039218*   
1                             Order  05B00039218   
First cell content: ''

Standard address table not found, initiating fallback search...

Searching Table 1 for address patterns...
Found 'Ship-to Address' at Table 1, Row 16, Column 2

Extracting from fallback position...

Final Address Values:
Ship-to: BRANDS Fashion GmbH
Agency: TRITON TEXTILE LIMITED Bangladesh
Buying House: Lager Buchholz
Müllerstr. 11
21244 B

In [ ]:
from babel.numbers import parse_decimal

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Amount USD': amount_usd
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    # Extract Amount value
    amount_match = re.search(r'Amount\s([\d\.,]+)', block)
    amount = amount_match.group(1) if amount_match else "0"

    # Convert European format to US float
    amount_usd = float(parse_decimal(amount, locale='de_DE'))

    # Store as dict
    orders.append({
        'Style No': style_no
    })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)


In [ ]:

# Capture each order block: from the first line until the line starting with Amount
order_blocks = re.findall(r'^(.*?Amount\s[\d\.,]+\s*\w*)', order_details_text, re.DOTALL | re.MULTILINE)

orders = []

for block in order_blocks:
    block = block.strip()
    lines = block.splitlines()
    print(f"Processing block:\n{block}\n")

    # Extract first word as Style No
    style_no = lines[0].split()[0] if lines else ""
    
    #Extract Customs no.
    customs_no_match = re.search(r'Customs\s+no\.?\s*:\s*([\d ]+)', block)
    if customs_no_match:
        customs_no = customs_no_match.group(1).strip()

    

    # Store as dict
    orders.append({
        'Style No': style_no,
        'Customs No': customs_no
        })

# Convert to DataFrame
df = pd.DataFrame(orders)
print(df)
